<a href="https://colab.research.google.com/github/pradee2601/Agentic_Al_Workshop/blob/main/HealthCare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-generativeai
!pip install gradio

# Import necessary libraries
import google.generativeai as genai
import os
import gradio as gr
from typing import Dict, List, Tuple
import json

In [2]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

model = genai.GenerativeModel(model_name='models/gemini-1.5-flash')
response = model.generate_content("Give me a fun fact about AI.")
print(response.text)

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2085.91ms


Some AI models are now being trained to generate not just text and images, but also completely novel scientific discoveries – proposing new materials with specific properties or even suggesting solutions to complex mathematical problems.  It's like having a super-powered, tireless lab assistant that's always brainstorming.



In [3]:
# Healthcare Policy Database
POLICY_DATABASE = {
    "Basic Health Plan": {
        "description": "Most affordable option for essential healthcare needs",
        "coverage": [
            "In-Patient Hospitalization (shared room)",
            "Emergency Room Services",
            "Basic Diagnostic Services (emergency-related only)",
            "Emergency Prescriptions (up to $250 annually)"
        ],
        "eligibility": "Individuals aged 18-55",
        "premium": "$150 per month",
        "deductible": "$1,500 per year",
        "coverage_limit": "$100,000",
        "network_hospitals": "2,500+",
        "exclusions": ["Outpatient services", "Wellness benefits", "Maternity", "Dental care"],
        "waiting_periods": {"pre_existing": "12 months"},
        "target_audience": ["young adults", "budget-conscious", "basic coverage"]
    },
    "Family Health Plus Plan": {
        "description": "Comprehensive coverage for families with in-patient and out-patient services",
        "coverage": [
            "Comprehensive In-Patient Hospitalization",
            "Family Doctor Visits (10 per year per member)",
            "Specialist Consultations (15 annually)",
            "Diagnostics and Lab Tests",
            "Emergency Care and Transport",
            "Prescription Drugs (up to $500 per member)"
        ],
        "eligibility": "Families, including dependent children up to age 25",
        "premium": "$350 per month",
        "deductible": "$1,000 per individual / $3,000 per family",
        "coverage_limit": "$500,000",
        "network_hospitals": "5,000+",
        "exclusions": ["Fertility treatments", "Cosmetic surgery", "Alternative therapies"],
        "waiting_periods": {"pre_existing": "6 months", "maternity": "9 months"},
        "target_audience": ["families", "children", "comprehensive coverage"]
    },
    "Comprehensive Health & Wellness Plan": {
        "description": "Holistic health support with wellness resources",
        "coverage": [
            "All Family Health Plus benefits",
            "Preventive Care Services",
            "Mental Health Support (6 sessions per year)",
            "Wellness Programs",
            "Additional Specialist Coverage",
            "Extended Prescription Drugs (up to $1,000 per member)"
        ],
        "eligibility": "Individuals and families with no age restrictions",
        "premium": "$500 per month",
        "deductible": "$750 per individual / $2,000 per family",
        "coverage_limit": "$1,000,000",
        "network_hospitals": "7,500+",
        "exclusions": ["Alternative therapies", "Experimental treatments"],
        "waiting_periods": {"pre_existing": "6 months", "mental_health": "3 months"},
        "target_audience": ["wellness-focused", "preventive care", "mental health"]
    },
    "Senior Health Security Plan": {
        "description": "Specialized coverage for seniors with extended healthcare needs",
        "coverage": [
            "Comprehensive Hospitalization with private rooms",
            "Unlimited Specialized Consultations",
            "Routine Prescriptions (up to $2,000 annually)",
            "Vision and Dental Benefits",
            "Senior Wellness Programs"
        ],
        "eligibility": "Individuals aged 55 and above",
        "premium": "$600 per month",
        "deductible": "$1,000 annually",
        "coverage_limit": "$750,000",
        "network_hospitals": "5,000+ with senior-focused facilities",
        "exclusions": ["Cosmetic treatments", "Experimental treatments"],
        "waiting_periods": {"pre_existing": "3 months"},
        "target_audience": ["seniors", "55+", "specialized care"]
    }
}

# Add-on options
ADD_ON_OPTIONS = {
    "Dental & Vision Add-On": {
        "cost": "$50 per month per member",
        "coverage": ["Two dental exams per year", "Annual eye exam", "$200 limit for corrective lenses"],
        "eligibility": "Available with all plans"
    },
    "Maternity & Newborn Care Add-On": {
        "cost": "$75 per month",
        "coverage": ["Pre-natal and post-natal care", "Delivery expenses", "Newborn immunizations"],
        "eligibility": "Only for Family Health Plus and Comprehensive Health & Wellness Plans",
        "waiting_period": "9 months"
    },
    "International Travel Medical Insurance": {
        "cost": "$40 per month",
        "coverage": ["Emergency medical coverage abroad up to $100,000", "24/7 global helpline"],
        "eligibility": "Available with all plans"
    }
}

print("✅ Policy database created successfully!")

✅ Policy database created successfully!


In [4]:
from typing import Dict, List, Tuple
import json

class HealthcarePolicyAgent:
    def __init__(self):
        self.model = model
        self.conversation_history = []

    def analyze_user_requirements(self, user_input: str) -> Dict:
        """Extract user requirements from input"""
        prompt = f"""
        Analyze the following user input and extract key information for healthcare policy recommendation:

        User Input: "{user_input}"

        Extract and return the following information in a structured format:
        - Age (if mentioned)
        - Family status (individual, family, number of dependents)
        - Special requirements (dental, vision, maternity, travel, wellness, mental health)
        - Budget considerations (if mentioned)
        - Specific health concerns (if mentioned)
        - Current health status or pre-existing conditions (if mentioned)

        Return this as a JSON-like structure with clear categories.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error analyzing requirements: {str(e)}"

    def recommend_policies(self, user_requirements: str) -> str:
        """Recommend suitable policies based on user requirements"""

        # Create policy information string
        policy_info = ""
        for plan_name, details in POLICY_DATABASE.items():
            policy_info += f"\n{plan_name}:\n"
            policy_info += f"- Description: {details['description']}\n"
            policy_info += f"- Eligibility: {details['eligibility']}\n"
            policy_info += f"- Premium: {details['premium']}\n"
            policy_info += f"- Coverage Limit: {details['coverage_limit']}\n"
            policy_info += f"- Key Coverage: {', '.join(details['coverage'][:3])}\n"

        prompt = f"""
        You are a healthcare insurance sales agent for HealthSecure Insurance Ltd.

        Based on the user's requirements: "{user_requirements}"

        And available policies: {policy_info}

        Please recommend the most suitable policy(ies) and explain why. Consider:
        1. Age and eligibility
        2. Family size and dependents
        3. Budget constraints
        4. Special health needs
        5. Coverage preferences

        Provide a clear recommendation with:
        - Primary policy recommendation with reasoning
        - Alternative options if applicable
        - Relevant add-ons that might be beneficial
        - Key benefits that match their needs
        - Next steps for enrollment

        Be conversational, helpful, and focus on value proposition.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating recommendation: {str(e)}"

    def handle_followup_question(self, question: str, context: str) -> str:
        """Handle follow-up questions about policies"""

        prompt = f"""
        You are a healthcare insurance sales agent. A user is asking a follow-up question about healthcare policies.

        Previous context: {context}

        User's follow-up question: "{question}"

        Available policy information: {json.dumps(POLICY_DATABASE, indent=2)}

        Available add-ons: {json.dumps(ADD_ON_OPTIONS, indent=2)}

        Please provide a helpful, accurate answer based on the HealthSecure Insurance policy details.
        Be specific about coverage, costs, eligibility, and any relevant policy details.
        """

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error handling follow-up: {str(e)}"

# Initialize the agent
agent = HealthcarePolicyAgent()
print("✅ Healthcare Policy Agent initialized!")

✅ Healthcare Policy Agent initialized!


In [5]:
import gradio as gr

def healthcare_policy_chatbot(message, history):
    """Main chatbot function with follow-up logic"""
    if not message.strip():
        return "📝 Please describe your healthcare insurance needs so I can assist you better!"

    last_context = history[-1][1] if history else ""

    if history and any(word in message.lower() for word in ['tell me more', 'what about', 'explain', 'how much', 'coverage', 'benefit']):
        return agent.handle_followup_question(message, last_context)

    return agent.recommend_policies(message)

def create_interface():
    """Create upgraded Gradio UI"""

    # Modern, clean CSS
    css = """
    .gradio-container {
        max-width: 900px !important;
        margin: 2rem auto !important;
        font-family: 'Segoe UI', sans-serif;
        background-color: #f7fafd;
        border-radius: 12px;
        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.05);
    }
    .prose {
        line-height: 1.6;
        font-size: 15px;
    }
    """

    # Create the Gradio ChatInterface
    demo = gr.ChatInterface(
        fn=healthcare_policy_chatbot,
        title="🏥 HealthCare - Insurance Policy Advisor",
        description="""
        <div class='prose'>
        Welcome to <strong>HealthCare Insurance</strong> 👋<br><br>
        I’ll help you find the best health insurance policy for your needs.<br><br>

        <strong>📌 What you can share with me:</strong>
        <ul>
            <li>👤 Your age and family status</li>
            <li>👨‍👩‍👧‍👦 Number of dependents (if any)</li>
            <li>🩺 Special needs (e.g. dental, maternity, vision)</li>
            <li>💰 Budget preferences</li>
            <li>📋 Any specific health conditions</li>
        </ul>

        <strong>📦 Sample Plans:</strong>
        <ul>
            <li><b>Basic Health Plan</b> – Rs 1500/month (Essential coverage)</li>
            <li><b>Family Health Plus</b> – Rs 3500/month (Full family support)</li>
            <li><b>Health & Wellness Plan</b> – Rs 5000/month (Wellness programs & more)</li>
            <li><b>Senior Health Security</b> – Rs 6000/month (Senior care focus)</li>
        </ul>

        Type below to get started!
        </div>
        """,
examples=[
    "I'm 30 and healthy, just need emergency coverage.",
    "Single parent with a child, looking for affordable family health insurance.",
    "Need a plan that covers mental health and online doctor consultations.",
    "We're both in our 50s, need a plan with specialist visits and prescription coverage.",
    "Young couple looking for maternity and newborn care options."
],

        theme=gr.themes.Soft(primary_hue="cyan"),
        css=css
    )

    return demo

# Launch
print("🚀 Launching HealthSecure Insurance Advisor UI...")
demo = create_interface()


🚀 Launching HealthSecure Insurance Advisor UI...


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [ ]:
# Launch the application
if __name__ == "__main__":
    # Launch with public link for sharing
    demo.launch(
        share=True,  # Creates a public link
        debug=True,  # Enable debug mode
        server_name="0.0.0.0",  # Allow external connections
        server_port=7860  # Default Gradio port
    )

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://610cd21038823423f0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
